#### Libraries

In [5]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import glob
import os
import re
import scipy.io
import csv
import sys
import time
import seaborn as sns

#### Pipeline to prepare data and get train/validation split
#### prepdata(filename, vidformat, chunklen, threshold=1, trainsplit=1, size=(192,108), csvformat=True, color=True, saveHQ=False, lowres=True)  
filename: video file name  
vidformat: format like avi/mov/mp4  
chunklen: number of frames/chunk of video  
threshold: min number of frames in a chunk where mosquito must be present to label as positive sample  
trainsplit: train_data / total_data  
size: resolution of saved data  
csvformat: True if csv, False if mat  
color: if True, color channels are retained, results in 5d data. Else data is 4d without color channels  
saveHQ: if True, full resolution images are also saved. Required for qualitative analysis of predictions  
lowres: if True, uses low resolution images to prepare data. Use to save memory  
#### Eg: prepdata('MVI_7500','.mov',5,1,0.8,color=False,csvformat=False,saveHQ=True)

In [12]:
# Extract frames from video and resize to lower resolution
# filename: video file name of .MOV format
# size: tuple(w*h)
# color: color or gray
# saveHQ: True if full res frames need to be saved
def vid2frames(filename, vidformat, size=(192,108), color=True, saveHQ=False):
    if not os.path.exists('Frames_resize/'+filename):
        os.makedirs('Frames_resize/'+filename)
        vidcap = cv2.VideoCapture('video/'+filename+vidformat)
        success,image = vidcap.read()
        count = 0
        while success:
            if color:
                img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            else:
                img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            img = (cv2.resize(img, size, interpolation=cv2.INTER_CUBIC))
            if saveHQ:
                if not os.path.exists('Frames/'+filename):
                    os.makedirs('Frames/'+filename)
                cv2.imwrite('Frames/'+filename+'/frame%d.jpg' % count, image)     # save frame as JPEG file
            cv2.imwrite('Frames_resize/'+filename+'/frame%d.jpg' % count, img)     # save frame as JPEG file 
            success,image = vidcap.read()
            count += 1
        print(count,'frames were generated')

# Load all gray/color images into a 4d array(num_blocks,5,h,w)/5d array(num_blocks,3,5,h,w)
# filename: video name
# chunklen: temporal dimension
# size: tuple(w*h)
# lowres: if True, uses the resized images
# color: whether or not to use color channels
def frames2np(filename, chunklen, size=(192,108), lowres=True, color=True):
    dim = 5 if color else 4
    quality = 'lq' if lowres else 'hq'
    if not os.path.isfile('data/train/'+filename+'_cl'+str(chunklen)+'_'+str(dim)+'d'+'_'+quality+'.npy'):        
        if lowres:
            path = 'Frames_resize/'+filename
        else:
            path = 'Frames/'+filename
        filelist = sorted(glob.glob(os.path.join(path,'*.jpg')),key=lambda stri: int(re.findall('\d+', stri)[0]))
        num_chunks = len(filelist)//chunklen
        filelist = filelist[:num_chunks*chunklen]
        if color:
            vid_np = np.zeros((num_chunks,chunklen,size[1],size[0],3))
        else:
            vid_np = np.zeros((num_chunks,chunklen,size[1],size[0]))
        i = 0
        j = 0
        for frame in filelist:
            if color:
                image = cv2.imread(frame, cv2.IMREAD_COLOR)
            else:
                image = cv2.imread(frame, cv2.IMREAD_GRAYSCALE)
            vid_np[i,j] = image
            j += 1
            if j % chunklen == 0:
                j = 0
                i += 1
        if color:
            vid_np = vid_np.reshape((num_chunks,3,chunklen,size[1],size[0]))
        np.save('data/train/'+filename+'_cl'+str(chunklen)+'_'+str(dim)+'d'+'_'+quality,vid_np)
    else:
        vid_np = np.load('data/train/'+filename+'_cl'+str(chunklen)+'_'+str(dim)+'d'+'_'+quality+'.npy')
    print('shape of numpy:',vid_np.shape)
    return vid_np.shape[0]

# generate 0 1 labels from framenums
# filename: video name
# chunklen: temporal dimension
# numchunks: data size
def framenum2labels(filename, chunklen, numchunks, threshold=1, csvformat=True):
    if not os.path.isfile('labels/train/'+filename+'_cl'+str(chunklen)+'.npy'):
        numframes = numchunks*chunklen
        if not csvformat:
            mat = scipy.io.loadmat('labels/'+filename+'.mat')
            try:
                framenums = np.array(mat['labels'][0])
            except:
                framenums = np.array(mat['frames'][0])
            framelabels = np.zeros(numframes)
            framelabels[framenums-1] = 1
        else:
            framelabels = []
            with open('labels/'+filename+'.csv') as csvfile:
                reader = csv.reader(csvfile)
                for row in reader:
                    framelabels.append(int(re.sub("\D", "", row[0])))
            framelabels = np.array(framelabels[:numframes])
        labels = 2*np.ones(numchunks)
        for i in range(chunklen,numframes+chunklen,chunklen):
            if np.sum(framelabels[i-chunklen:i]) > threshold-1:
                labels[i//chunklen-1] = 1
            else:
                labels[i//chunklen-1] = 0
        if 2 in labels:
            sys.exit('Error: The labels contain label 2')
        labels = np.column_stack((labels,np.arange(numchunks)))
        np.save('labels/train/'+filename+'_cl'+str(chunklen),labels)
    else:
        labels = np.load('labels/train/'+filename+'_cl'+str(chunklen)+'.npy')
    print('shape of labels:',labels.shape)

# split the data into training and validation sets
# filename: numpy file name without d
# trainsplit: portion of training data
# dim: 4 or 5
def split(filename,trainsplit,dim,quality):
    npfile = np.load('data/train/'+filename+'_'+str(dim)+'d'+'_'+quality+'.npy')
    label = np.load('labels/train/'+filename+'.npy')
    if trainsplit < 1 and trainsplit > 0:
        size = len(npfile)
        trainbatch = int(size*trainsplit)
        indices = np.random.permutation(size)
        npfile = npfile[indices]
        label = label[indices]
        train_data = npfile[:trainbatch]
        train_labels = label[:trainbatch]
        validation_data = npfile[trainbatch:]
        validation_labels = label[trainbatch:]
        np.save('data/train/'+filename+'_'+str(dim)+'d_'+quality+'_'+str(trainsplit), train_data)
        np.save('data/validation/'+filename+'_'+str(dim)+'d_'+quality+'_'+str(trainsplit), validation_data)
        np.save('labels/train/'+filename+'_'+str(trainsplit),train_labels)
        np.save('labels/validation/'+filename+'_'+str(trainsplit),validation_labels)      
        print('train shape:',train_data.shape, 'validation shape:',validation_data.shape)
        print('numpy file:',filename+'_'+str(dim)+'d_'+quality+'_'+str(trainsplit))
        print('label file:',filename+'_'+str(trainsplit))

# runs the video and label throught he pipeline to produce training and validation datasets
# by default, no validation dataset is generated
# csv: True if the labels are in a csv
def prepdata(filename, vidformat, chunklen, threshold=1, trainsplit=1, size=(192,108), csvformat=True, color=True, saveHQ=False, lowres=True):
    vid2frames(filename, vidformat=vidformat, size=size, color=color, saveHQ=saveHQ)
    numchunks = frames2np(filename, chunklen, size=size, lowres=lowres, color=color)
    framenum2labels(filename, chunklen, numchunks, threshold=threshold, csvformat=csvformat)
    if trainsplit != 1:
        filename = filename+'_cl'+str(chunklen)
        dim = 5 if color else 4
        quality = 'lq' if lowres else 'hq'
        split(filename,trainsplit,dim,quality)

#### combine datasets and divide train and validation sets
#### appenddata(filename1,filename2,chunklen,dim,quality,trainsplit=1)  
filename1:  
filename2:  
chunklen: number of frames/chunk of video  
dim: dimensions of data. 4 without color channels, 5 with color channels  
quality: quality of data. 'lq' for low resolution, 'hq' for high resolution  
trainsplit: train_data / total_data  
#### Eg: appenddata('MVI_7500','MVI_7503',5,4,'lq',0.8)

In [24]:
def appenddata(filename1,filename2,chunklen,dim,quality,trainsplit=1):
    if trainsplit != 1:
        file1 = np.load('data/train/'+filename1+'_cl'+str(chunklen)+'_'+str(dim)+'d_'+quality+'_'+str(trainsplit)+'.npy')
        file2 = np.load('data/train/'+filename2+'_cl'+str(chunklen)+'_'+str(dim)+'d_'+quality+'_'+str(trainsplit)+'.npy')
        label1 = np.load('labels/train/'+filename1+'_cl'+str(chunklen)+'_'+str(trainsplit)+'.npy')
        label2 = np.load('labels/train/'+filename2+'_cl'+str(chunklen)+'_'+str(trainsplit)+'.npy')
        validation1 = np.load('data/validation/'+filename1+'_cl'+str(chunklen)+'_'+str(dim)+'d_'+quality+'_'+str(trainsplit)+'.npy')
        validation2 = np.load('data/validation/'+filename2+'_cl'+str(chunklen)+'_'+str(dim)+'d_'+quality+'_'+str(trainsplit)+'.npy')
        vlabel1 = np.load('labels/validation/'+filename1+'_cl'+str(chunklen)+'_'+str(trainsplit)+'.npy')
        vlabel2 = np.load('labels/validation/'+filename2+'_cl'+str(chunklen)+'_'+str(trainsplit)+'.npy')
        validation_set = np.append(validation1, validation2, axis=0)
        np.save('data/validation/'+filename1+'_'+filename2+'_cl'+str(chunklen)+'_'+str(dim)+'d_'+quality+'_'+str(trainsplit),validation_set)
        np.save('labels/validation/'+filename1+'_'+filename2+'_cl'+str(chunklen)+'_'+str(trainsplit),np.append(vlabel1, vlabel2, axis=0))
    else:
        file1 = np.load('data/train/'+filename1+'_cl'+str(chunklen)+'_'+str(dim)+'d_'+quality+'.npy')
        file2 = np.load('data/train/'+filename2+'_cl'+str(chunklen)+'_'+str(dim)+'d_'+quality+'.npy')
        label1 = np.load('labels/train/'+filename1+'_cl'+str(chunklen)+'.npy')
        label2 = np.load('labels/train/'+filename2+'_cl'+str(chunklen)+'.npy')
    train_set = np.append(file1, file2, axis=0)
    np.save('data/train/'+filename1+'_'+filename2+'_cl'+str(chunklen)+'_'+str(dim)+'d_'+quality+'_'+str(trainsplit),train_set)
    np.save('labels/train/'+filename1+'_'+filename2+'_cl'+str(chunklen)+'_'+str(trainsplit),np.append(label1, label2, axis=0))
    print('train shape:',train_set.shape, 'validation shape:',validation_set.shape)
    print('numpy file:',filename1+'_'+filename2+'_cl'+str(chunklen)+'_'+str(dim)+'d_'+quality+'_'+str(trainsplit))
    print('label file',filename1+'_'+filename2+'_cl'+str(chunklen)+'_'+str(trainsplit))

#### annotate videos  
#### annotate(video, vidformat, x=1, save=True, showtimes=False)  
video: name of video  
vidformat: format like avi/mov/mp4  
x: speed  
save: save generated labels  
showtimes: show timestamps when mosquito present
#### return: generated labels  
#### Eg: annotate('MVI_7500','mov',showtimes=True)

In [8]:
def annotate(video, vidformat, x=1, save=True, showtimes=False):

    def onSlide(pos):
        slider_position[0] = pos
        cap.set(cv2.CAP_PROP_POS_FRAMES,pos)

    def prev_next(action, current_state):
        end = False
        if action == ord('p'):
            pass
        elif action == ord('d'):
            ret, frame = cap.read()
            if not ret:
                end = True
                return end, current_state
            cv2.imshow('frame',frame)
            slider_position[0] += 1
            cv2.setTrackbarPos('position', 'frame', slider_position[0])
            annotation_list[int(cap.get(cv2.CAP_PROP_POS_FRAMES))-1] = current_state
            action = cv2.waitKey(0)
            end, current_state = prev_next(action, current_state)
        elif action == ord('a'):
            onSlide(slider_position[0]-2)
            ret, frame = cap.read()
            cv2.imshow('frame',frame)
            slider_position[0] += 1
            cv2.setTrackbarPos('position', 'frame', slider_position[0])
            action = cv2.waitKey(0)
            end, current_state = prev_next(action, current_state)
        elif action == ord(' '):
            current_state = not current_state
            print('Current state',current_state)
            annotation_list[int(cap.get(cv2.CAP_PROP_POS_FRAMES))-1] = current_state
            action = cv2.waitKey(0)
            end, current_state = prev_next(action, current_state)
        return end, current_state

    current_state = False
    slider_position = [0]
    cv2.namedWindow('frame',cv2.WINDOW_NORMAL)
    cap = cv2.VideoCapture('video/'+video+'.'+vidformat)
    frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    annotation_list = np.zeros(frames)
    if frames != 0:
        cv2.createTrackbar('position', 'frame', slider_position[0], frames, onSlide)
    while(True):
        ret, frame = cap.read()
        if not ret:
            break
        cv2.imshow('frame',frame)
        slider_position[0] += 1
        cv2.setTrackbarPos('position', 'frame', slider_position[0])
        annotation_list[int(cap.get(cv2.CAP_PROP_POS_FRAMES))-1] = current_state
        key = cv2.waitKey(33//x)

        if key & 0xFF == ord(' '):
            current_state = not current_state
        if key & 0xFF == ord('p'):
            action = cv2.waitKey(0)
            end, current_state = prev_next(action, current_state)
            if end:
                break
        if key == 27:#if ESC is pressed, exit loop
            cv2.destroyAllWindows()
            break
    cap.release()
    cv2.destroyAllWindows()
    annotation_list = annotation_list.astype(int)
    if save:
        np.savetxt('labels/'+video+'.csv',annotation_list,delimiter=',',fmt='%d')
    txt = ''
    for i in range(0, frames, 30):
        if sum(annotation_list[i:i+30]):
            txt += '('+str(i//1800)+':'+str((i//30)%60)+') // '
    with open('labels/'+video+'.txt', "w") as text_file:
        text_file.write(txt)
    if showtimes:
        print(txt)
    return annotation_list

In [11]:
prepdata('MVI_7500','.mov',5,1,0.8,color=False,csvformat=True,saveHQ=True)

1 0.8
shape of numpy: (629, 5, 108, 192)
shape of labels: (629, 2)
train shape: (503, 5, 108, 192) validation shape: (126, 5, 108, 192)
numpy file: MVI_7500_cl5_4d_lq_0.8
label file: MVI_7500_cl5_0.8


In [14]:
prepdata('MVI_7503','.mov',5,1,0.8,color=False,csvformat=True,saveHQ=True)

4662 frames were generated
shape of numpy: (932, 5, 108, 192)
shape of labels: (932, 2)
train shape: (745, 5, 108, 192) validation shape: (187, 5, 108, 192)
numpy file: MVI_7503_cl5_4d_lq_0.8
label file: MVI_7503_cl5_0.8


In [15]:
prepdata('MVI_7512','.mov',5,1,0.8,color=False,csvformat=True,saveHQ=True)

2991 frames were generated
shape of numpy: (598, 5, 108, 192)
shape of labels: (598, 2)
train shape: (478, 5, 108, 192) validation shape: (120, 5, 108, 192)
numpy file: MVI_7512_cl5_4d_lq_0.8
label file: MVI_7512_cl5_0.8


In [16]:
prepdata('MVI_7507','.mov',5,1,0.8,color=False,csvformat=True,saveHQ=True)

9389 frames were generated
shape of numpy: (1877, 5, 108, 192)
shape of labels: (1877, 2)
train shape: (1501, 5, 108, 192) validation shape: (376, 5, 108, 192)
numpy file: MVI_7507_cl5_4d_lq_0.8
label file: MVI_7507_cl5_0.8


In [19]:
appenddata('MVI_7500','MVI_7503',5,4,'lq',0.8)

numpy file: MVI_7500_MVI_7503_cl5_4d_lq_0.8
label file MVI_7500_MVI_7503_cl5_0.8


In [20]:
appenddata('MVI_7500_MVI_7503','MVI_7512',5,4,'lq',0.8)

numpy file: MVI_7500_MVI_7503_MVI_7512_cl5_4d_lq_0.8
label file MVI_7500_MVI_7503_MVI_7512_cl5_0.8


In [25]:
appenddata('MVI_7500_MVI_7503_MVI_7512','MVI_7507',5,4,'lq',0.8)

train shape: (3227, 5, 108, 192) validation shape: (809, 5, 108, 192)
numpy file: MVI_7500_MVI_7503_MVI_7512_MVI_7507_cl5_4d_lq_0.8
label file MVI_7500_MVI_7503_MVI_7512_MVI_7507_cl5_0.8


In [26]:
appenddata('MVI_7500_MVI_7503','MVI_7507',5,4,'lq',0.8)

train shape: (2749, 5, 108, 192) validation shape: (689, 5, 108, 192)
numpy file: MVI_7500_MVI_7503_MVI_7507_cl5_4d_lq_0.8
label file MVI_7500_MVI_7503_MVI_7507_cl5_0.8


In [27]:
framelabels = annotate('MVI_7507','mov',x=2,showtimes=True)

Current state True
Current state False
(0:5) // (0:6) // (0:7) // (0:8) // 


In [73]:
framelabels[-10:]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [54]:
ord('p')

112

In [68]:
txt='(0:2) // (0:3) // (0:8) // (0:13) // (0:19) // (0:20) // (0:24) // (0:25) // (0:26) // (0:28) // (0:29) // (0:30) // (0:54) // (0:55) // (0:56) // (1:5) // (1:6) // (1:8) // (1:9) // (1:10) // (1:11) // (1:12) // (1:13) // (1:14) // (1:22) // (1:23) // (1:24) // (1:25) // (1:26) // (1:27) // (1:28) // (1:32) // (1:33) // (1:34) // (1:37) // (1:38) // (1:39) // (1:40) // (1:41) // (1:42) // (1:43) // (1:44) // '
with open("labels/MVI_7500.txt", "w") as text_file:
    text_file.write(txt)